In [12]:
import pandas as pd
import ee
import geetools
import folium
import time
from folium import plugins
import Leaf_tools
import math
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AY0e-g64UBVFCZnGAlm3wqqf7Dnj_drCsQ_es88hzz4fIvpc9DsFme7IZo4



Successfully saved authorization token.


**Import data**

In [14]:
tiles_CCRS = ee.FeatureCollection("users/ganghong/TilesSampling_Sub")
print (tiles_CCRS.size().getInfo())
tiles_Bel = ee.FeatureCollection('users/ganghong/TilesSampling_Belmanip_Sub')
print (tiles_Bel.size().getInfo())
ezone = ee.FeatureCollection("users/ganghong/NA_Forests")
fapar_CCRS = ee.ImageCollection("users/ganghong/FAPAR_CCRS")
fcover_CCRS = ee.ImageCollection("users/ganghong/FCOVER_CCRS")
LAI_CCRS = ee.ImageCollection("users/ganghong/LAI_CCRS")
fapar_Bel = ee.ImageCollection("users/ganghong/FAPAR_Bel")
fcover_Bel = ee.ImageCollection("users/ganghong/FCOVER_Bel")
LAI_Bel = ee.ImageCollection("users/ganghong/LAI_Bel")
##Define Forest Cover map to use - should be a global constant
NALCMS_forest = ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles')\
.map(lambda image:image.select("b1").rename("landcover"))\
.merge(ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global")\
.map(lambda image:image.select("discrete_classification").remap([0,20,30,40,50,60,70,80,90,100,111,112,113,114,115,116,121,122,123,124,125,126,200],[0,8,10,15,17,16,19,18,14,13,1,3,1,5,6,6,2,4,2,5,6,6,18],0).toUint8().rename("landcover")))

# print (fapar_CCRS.size().getInfo())
# print (fcover_CCRS.size().getInfo())
# print (LAI_CCRS.size().getInfo())
# print (fapar_Bel.size().getInfo())
# print (fcover_Bel.size().getInfo())
# print (LAI_Bel.size().getInfo())

28
35


**Functions definition**

In [15]:
basemaps = {
'Google Maps': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Maps',
overlay = True,
control = True
),
'Google Satellite': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Satellite',
overlay = True,
control = True
)}

## display layers
def add_ee_layer(self, ee_object, vis_params, name):
 try:
  if isinstance(ee_object, ee.image.Image):
   map_id_dict = ee.Image(ee_object).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.imagecollection.ImageCollection):
   ee_object_new = ee_object.mosaic()
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name, 
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.geometry.Geometry):
   folium.GeoJson(
   data = ee_object.getInfo(),
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object,ee.featurecollection.FeatureCollection):
   ee_object_new = ee.Image().paint(ee_object, 0, 2)
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

 except:
  print("Could not display {}".format(name))

#date conversion
def DatetoSystemtime(datelist):
    geestamp=[]
    for date in datelist:       
        date = pd.to_datetime(date, format='%Y-%m-%d')       
       # print (date)
        year_day = pd.Timestamp(year=date.year, month=1, day=1)       
        doy = (date - year_day).days + 1     ## add three digits ###   
        doy='{:>03}'.format(doy) ### three digit format for doy 000
        hour = '12'
        year=date.year
        timestamp = str(year) + doy + hour
        # convert time stamp to gee format
        gee_timestamp = int(pd.to_datetime(timestamp,format='%Y%j%H').value // 10**6)
        #print (gee_timestamp)
        geestamp.append(gee_timestamp)        
    return geestamp    

#remove properties 
def removeProperty(feat): 
    properties = feat.propertyNames()
    selectProperties = properties.filter(ee.Filter.eq('item', 'MGRS_TILE')) 
    return feat.select(selectProperties)

#featurecollection intersection
def featurecollection_interset(tile, resultlist):
    resultlist = ee.List(resultlist)
    tile = ee.Feature(tile)
    def polygonintersect(zone):    
        zone = ee.Feature(zone)   
        intersection = tile.intersection(zone, ee.ErrorMargin(1));
        return intersection.set({'ezone': zone.get('EZMAP_ZONE')}).set({'Intersect': intersection.area().divide(1000 * 1000)})
    intersection = zoneList.map(polygonintersect)
    sortedintersect=ee.FeatureCollection(intersection).sort('Intersect', False)
    return resultlist.add(sortedintersect.first())

#land cover frequency vector based
def calculateFreq(tile,list):
    list=ee.List(list)
    frequency = NALCMS_forest.mosaic().reduceRegion(
       reducer=ee.Reducer.frequencyHistogram(),
       maxPixels=2e9,
       scale=500,
       geometry=tile.geometry()
      )
    tempobj= frequency.get("landcover")
    okeys = ee.Dictionary(tempobj).keys()
    ovals = ee.Dictionary(tempobj).values()
    minmax = ee.Dictionary(ovals.reduce(ee.Reducer.minMax()))
    maxval = ee.Number(minmax.get('max'))
    maxind = ovals.indexOf(maxval)
    maxkey = okeys.get(maxind)
    tile= tile.set({'landcover': maxkey})
    return list.add(tile)

#feature to dataframe
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']
    dictarr = []
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        #attr['geometry'] = f['geometry']  # GeoJSON Feature!
        # attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)
        df = GeoDataFrame(dictarr)
        # Convert GeoJSON features to shape
        #df['geometry'] = map(lambda s: shape(s), df.geometry)
    return df

#add date to a feature
def datelist(img):
    imgdate=ee.Date(img.get("system:time_start"))
    return ee.Feature(None, {'date': imgdate.format('YYYY-MM-dd')})
 
#bit wsie operation  
def bitwiseExtract(value, fromBit, toBit):
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)

#modis product bit mask
def bitmask(image):
    qa = image.select('FparLai_QC')
    good = bitwiseExtract(qa, 0,0) # returns 0 for good quality
    return image.updateMask(ee.Image(good).eq(0))  ## needs to be inverted to mask pixels with bad quality, 0 means good quality , pass the mask  

## function for calculate raster dominant 
def dominantCal(raster, coarseProj, coarseScale, targetscale, minVal, maxVal, steps):
    ##calcualte stats from reduceResolution
    stats = raster.reproject(raster.projection(), None, targetscale).reduceResolution(
       reducer= ee.Reducer.fixedHistogram(minVal, maxVal, steps),
       maxPixels= 65535,
       bestEffort= True
    ).reproject(
      crs=coarseProj,
      scale= coarseScale##MODISscale
    )

    array=stats.toArray()
    ## slice to land cover
    landcover=array.arraySlice(1,0,1)
    ##slice to percentage
    percentage=array.arraySlice(1,1,2)
    ##sort percentage
    sortedper = array.arraySort(percentage);
    ## get values
    length = sortedper.arrayLength(0)
    values = sortedper.arraySlice(0, length.subtract(1), length);
    ##convert back to an image  
    maxpercentage = values.arrayProject([1]).arrayFlatten([['landcover', 'percentage']])
    dominantlc=maxpercentage.select('landcover').ceil()
    return maxpercentage.select('landcover').ceil()

## function for add dominat lc and ez
def addDZ(dominantEZ,dominantLC):
    def wrap(img):
        return img.addBands(dominantEZ.rename('dominantEZ')).addBands(dominantLC.rename('dominantLC'))
    return wrap

 ## Extract the ID of each image 
def  getidList(item):
     return ee.Image(item).id()
 
 ## matched date between sensors   
def datematched(halftempRes,dataMODIS):
    def wrap(img):
            dateImg=img.get('date')
            startdate= ee.Date(dateImg).advance(-halftempRes,'day')  
            enddate= ee.Date(dateImg).advance(halftempRes,'day')
            dataMODIS2 = dataMODIS.filterDate(startdate, enddate)  ## dataMODIS is a default reference sensor
            size=dataMODIS2.size()
            return img.set('matchedData',ee.Algorithms.If(size.eq(0),'none', dataMODIS2.toList(size)))
    return wrap

##the function to get the corase resoltuion image collection with matched sensor information 
## startDate, endDate: dates to start and end
##mapBounds: tile boundary 
## tilename: MGRS tile used to filter data, used in S2 only
##parameter: LAI, fAPAR,fCOVER,make sure correponding to name definition leaf-tool box
##tempRes: temporal resolution, like 5 days, 7 days
##coarsescale: scale used for getting the coarse resolution
## MODIS_paraBand, MODIS_un: names of MODIS parameter and uncertainty bands
## S2_paraBand, S2_un: names of S2 parameter and uncertianty bands
## CLGS_parameter: name of parameter Collection, LAI_CCRS in the import data section
## CLGS_paraBand, CLGS_un, CLGS_flag: names of CLGS parameter , uncertianty and quality flag bands
def applySensorComp(startDate, endDate, mapBounds, tilename, parameter, tempRes, coarsescale, MODIS_paraBand, MODIS_un, S2_paraBand, S2_un, CLGS_parameter, CLGS_paraBand, CLGS_un, CLGS_flag):
    
    dataMODIS =ee.ImageCollection('MODIS/006/MCD15A3H').filterBounds(mapBounds).filterDate(starDate, endDate)
    dataS2=Leaf_tools.Leaf_tools(mapBounds,starDate,endDate,tilename,parameter)#.filter(ee.Filter.eq('MGRS_TILE', tilename))
    coarsecrs=dataMODIS.first().projection()
   # coarsescale=dataMODIS.first().projection().nominalScale()
    para_un=parameter+'un' ## 'LAIun'
    ##update mask and set a new property date
    dataMODIS=dataMODIS.map(validMODISMask)##lambda img:img.updateMask(img.select('FparLai_QC').eq(0)))  ## bit 0: 0 which means Good quality (main algorithm with or without saturation)
    dataMODIS=dataMODIS.map(lambda img:img.select([MODIS_paraBand, MODIS_un],[parameter, para_un]).set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")))   #['Lai','LaiStdDev'],['LAI','LAIun']
    dataCLGS=CLGS_parameter.filterBounds(mapBounds).filterDate(starDate, endDate).map(validCLGSMask(CLGS_flag))##lambda img:img.unmask().updateMask(img.select(CLGS_flag).eq(1))) ###bit 0 equal to 1 means land
    dataCLGS=dataCLGS.map(lambda img:img.select([CLGS_paraBand,CLGS_un],[parameter, para_un]).set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd"))) 
    dataS2=dataS2.map(lambda img:img.updateMask(img.select('QC').eq(0)))
    dataS2=dataS2.map(lambda img:img.select([S2_paraBand,S2_un],[parameter, para_un]).set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")))
    
    ### date matching    
    halftempRes=math.ceil(tempRes/2)
    ##print (halftempRes)
    dataS2matched= dataS2.map(datematched(halftempRes,dataMODIS))    
    dataCLGSmatched= dataCLGS.map(datematched(halftempRes,dataMODIS))
    
    ### coarse resolution reducers  
    reducers = ee.Reducer.mean().combine(
      reducer2= ee.Reducer.stdDev(),
      sharedInputs= True
    ).combine(
      reducer2= ee.Reducer.count(),
      sharedInputs= True
    )
    ##coarse CLGS-product1
    coarseCLGS=dataCLGSmatched.map(lambda img:img.reduceResolution(
               reducer= reducers,
               maxPixels= 65535,
               bestEffort= True
               ).reproject(
                   crs= coarsecrs,
                   scale= coarsescale
                 ))
    ##coarse S2-product2
    coarseS2=dataS2matched.map(lambda img:img.reproject(dataS2matched.first().projection(), None, 20).reduceResolution(
               reducer= reducers,
               maxPixels= 65535,
               bestEffort= True
               ).reproject(
                   crs= coarsecrs,
                   scale= coarsescale
                 ))
    #### coarse MODIS product
    coarseMODIS=dataMODIS.map(lambda img:img.reduceResolution(
               reducer= reducers,
               maxPixels= 65535,
               bestEffort= True
               ).reproject(
                   crs= coarsecrs,
                   scale= coarsescale
                 ))

    ### dominant land cover
    tilelandcover=NALCMS_forest.filterBounds(mapBounds).mosaic().clip(mapBounds)  
    dominantLC=dominantCal(tilelandcover, coarsecrs,  coarsescale, 90, 0, 19, 19) ##(0, 19, 19)
    
    ## convert ezone ploygon to raster
    ezraster=ee.FeatureCollection(ezone).reduceToImage(['EZMAP_CODE'],ee.Reducer.first( )).clip(mapBounds)
    tileez=ezraster.clip(mapBounds)
     ### dominant ezone
    dominantEZ=dominantCal(tileez, coarsecrs, coarsescale, 200, 0, 99, 99) ##(0, 99, 99)
 
    ## add dominant land cover,ezone
    coarseCLGSwithDominant=coarseCLGS.map(addDZ(dominantEZ,dominantLC))   
     ## add dominant land cover,ezone
    coarseS2withDominant=coarseS2.map(addDZ(dominantEZ,dominantLC))
    ## add dominant land cover,ezone
    coarseMODISwithDominant=coarseMODIS.map(addDZ(dominantEZ,dominantLC))
    
    ### return three coarse image collection, the matched MODIS data information include the property matchedData, 
    ###but the images information included is still orignal MODIS images, 
    ##it could be updated through getting system id from original MODIS and use ee.Filter.inList to find the corresponding images in coarse MODIS collection.    
    return coarseS2withDominant, coarseCLGSwithDominant,coarseMODISwithDominant  

## mask for CLGS data  CLGS_flag
def validCLGSMask(CLGS_flag):
    def wrap(image):
        qa = image.select(CLGS_flag)   ## CLGS has different bands for each parameter, like b3 for LAI, b2 for FAPAR
        land = qa.bitwiseAnd(1 << 0)   ## bit 0 equal to 1, land 
        return image.unmask().updateMask(land)  ##bit 0 equal to 1 means land ; Goolge Earth Engine masked no-data already
    return wrap


## mask for MODIS data
def validMODISMask(image):
    qa = image.select('FparLai_QC')
    qaBitsGood = qa.bitwiseAnd(1 << 0) ##bit 0, 0 mean good , 1 bad pixel
    validmask=qaBitsGood.eq(0) 
    return image.updateMask(validmask)


**Run functions**

In [16]:
starDate='2019-04-01'
endDate='2019-09-30'
mapBounds= ee.FeatureCollection(ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(0))).geometry()
tilename=ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(0)).get('MGRS_TILE')
print('MGRS tilename:',tilename.getInfo())
## run function applySensorComp
coarseS2_output, coarseCLGS_ouput,coarseMODIS_output=applySensorComp('2019-04-01', '2019-09-30', mapBounds, tilename, 'LAI', 5, 500, 'Lai','LaiStdDev', 'estimateLAI','errorLAI', LAI_CCRS,'b1','b2','b3')
print (coarseS2_output.size().getInfo(), 'coarseS2_output')
print (coarseCLGS_ouput.size().getInfo(), 'coarseCLGS_ouput')
print (coarseMODIS_output.size().getInfo(), 'coarseMODIS_output')

print ("########################################DEMO for extract matched images between sensors##################################")

print ("########################################CLGS##############################################################################")
imgList_CLGSdominant = coarseCLGS_ouput.toList(coarseCLGS_ouput.size());
print('size of imageList_CLSGS:', imgList_CLGSdominant.size().getInfo());
    #props_CLGSdominant = ee.Image(imgList_CLGSdominant.get(0)).toDictionary(ee.Image(imgList_CLGSdominant.get(0)).propertyNames())
    #print (props_CLGSdominant.getInfo())
bands_CLGSdominant = ee.Image(imgList_CLGSdominant.get(0)).bandNames()
print ('bands name:',bands_CLGSdominant.getInfo())

MODISlist_CLGSdominant=ee.Image(imgList_CLGSdominant.get(0)).get('matchedData')
print ('matched MODIS_CLGS: ',ee.List(MODISlist_CLGSdominant).getInfo())
print ('size of matched MODIS_CLGS:  ',ee.List(MODISlist_CLGSdominant).size().getInfo(), )
    #print (imageList.get(0), 'Result index 0 ')
    #print (ee.Image(imageList.get(0)).get('matchedData'), 'matched MODIS')
MODISlist_CLGS=ee.List(MODISlist_CLGSdominant).map(getidList)
print ('MODISlist_CLGS sytem id list:',MODISlist_CLGS.getInfo())
    ### get the corresponding coarse MODIS
coarseMODISList_CLGS= coarseMODIS_output.filter(ee.Filter.inList('system:index', MODISlist_CLGS)) ## use the sytem index to locate coarse MODIS
print ('coarseMODISList_CLGS band names: ',ee.Image(coarseMODISList_CLGS.toList(coarseMODISList_CLGS.size()).get(0)).bandNames().getInfo())
print ("########################################S2################################################################################")
imgList_S2dominant = coarseS2_output.toList(coarseS2_output.size()) # convert colleciton to list
print('size of imageList_S2:  ', imgList_S2dominant.size().getInfo());
#props_S2dominant = ee.Image(imgList_S2dominant.get(0)).toDictionary(ee.Image(imgList_S2dominant.get(0)).propertyNames())
#print (props_S2dominant.getInfo())
bands_S2dominant = ee.Image(imgList_S2dominant.get(0)).bandNames() ## get the band names of  first image of the list
print ('band names:  ',bands_S2dominant.getInfo())

MODISlist_S2dominant=ee.Image(imgList_S2dominant.get(0)).get('matchedData') ## get the property 'matchedData' of  first image of the list
print ('matched MODIS_S2:  ', MODISlist_S2dominant.getInfo())
print ('size of matched MODIS_S2:  ',ee.List(MODISlist_S2dominant).size().getInfo())  # size of the property 'matchedData' 
#print (imageList.get(0), 'Result index 0 ')
#print (ee.Image(imageList.get(0)).get('matchedData'), 'matched MODIS')

MODISlist_S2=ee.List(MODISlist_S2dominant).map(getidList) ## get the ids of images in the property 'matchedData'
print ('MODISlist_S2 sytem id list:  ',MODISlist_S2.getInfo())
### get the corresponding coarse MODIS
## use the sytem index to locate coarse MODIS since the matching was occured before the coarse resolution, the image in the matched property is original images
coarseMODISList_S2= coarseMODIS_output.filter(ee.Filter.inList('system:index', MODISlist_S2)) 
print ('coarseMODISList_S2 band name:',ee.Image(coarseMODISList_S2.toList(coarseMODISList_S2.size()).get(0)).bandNames().getInfo())  


MGRS tilename: 09WXP
85 coarseS2_output
16 coarseCLGS_ouput
45 coarseMODIS_output
########################################DEMO for extract matched images between sensors##################################
########################################CLGS##############################################################################
size of imageList_CLSGS: 16
bands name: ['LAI_mean', 'LAI_stdDev', 'LAI_count', 'LAIun_mean', 'LAIun_stdDev', 'LAIun_count', 'dominantEZ', 'dominantLC']
matched MODIS_CLGS:  [{'type': 'Image', 'bands': [{'id': 'LAI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [86400, 43200], 'crs': 'SR-ORG:6974', 'crs_transform': [463.312716528, 0, -20015109.354, 0, -463.312716527, 10007554.677]}, {'id': 'LAIun', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [86400, 43200], 'crs': 'SR-ORG:6974', 'crs_transform': [463.312716528, 0, -20015109.354, 0, -463.312716527, 10007554.677]}], 'versio

**test Fapar**

In [17]:
coarseS2_Fapar_output, coarseCLGS_Fapar_ouput,coarseMODIS_Fapar_output=applySensorComp('2019-04-01', '2019-09-30', mapBounds, tilename, 'fAPAR', 5, 500,'Fpar','FparStdDev', 'estimatefAPAR','errorfAPAR', fapar_CCRS,'b6','b1','b2')
print (coarseS2_Fapar_output.size().getInfo(), 'coarseS2_Fapar_output')
print (coarseCLGS_Fapar_ouput.size().getInfo(), 'coarseCLGS_Fapar_ouput')
print (coarseMODIS_Fapar_output.size().getInfo(), 'coarseMODIS_Fapar_output')

85 coarseS2_Fapar_output
17 coarseCLGS_Fapar_ouput
45 coarseMODIS_Fapar_output
